In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()


sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)

print(output)
print(stemmer.stem('Mereka meniru-nirukannya'))

In [ ]:
from pattern3.web import Twitter, URL = 
from tqdm import tqdm

def crawl(topic, N=100, Nbatch=25, fullTalk=False, language = 'id'):
    t = Twitter(language = 'id')
    M = N//Nbatch
    i, Tweets, keepCrawling = None, [], True
    for j in tqdm(range(int(M))):
        if keepCrawling:
            for tweet in t.search(topic, language=language, start=i, count=Nbatch):
                try:
                    Tweets.append(tweet)
                    i = tweet.id
                except:
                    print("Twitter Limit reached")
                    keepCrawling = False 
                    break
                else:
                    break
            print('Making sure we get the full tweets, please wait....')
            for i, tweet in enumerate(tqdm(Tweets)):
                try:
                    webPage = URL(tweet.url).download()
                    soup = bs(webPage,'html parser')
                    full_tweet = soup.find_all('p', class_='TweetTextSize')
                    if fullTalk:
                        T = []
                        for talk in full_tweet:
                            T.append(bs(str(talk), 'html.parser').text)
                        full_tweet=' \n'.join(T)
                    else:
                        full_tweet=bs(str(full_tweet[0]), 'html.parser').text
                    Tweets[i]['fullTxt']=full_tweet
                except:
                    Tweets[i]['fullTxt']=tweet.txt
            print('Done.... Total terdapat {0} tweet'.format(len(Tweets)))
            return Tweets

            topic = 'song jong ki'
            Tweets = crawl(topic, N=30)
            return Tweets

In [ ]:
for twit in Tweets:
    print("{}".format(twit))

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re, networkx as nx, matplotlib.pyplot as plt, numpy as np 
from nltk.tokenize import TweetTokenizer; Tokenizer = TweetTokenizer(reduce_len=True)

def cleanTweets(Tweets):
    factory = StopWordRemoverFactory(); stopwords = set(factory.get_stop_words()+['twitter', 'rt', 'pic', 'com', 'yg', 'ga', 'httpd'])
    factory = StemmerFactory(); stemmer = factory.create_stemmer()
    for i, tweet in enumerate(tqdm(Tweets)):
        txt = tweet['fullTxt']
        txt = re.sub(r'http[s]?://([a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',txt)
        txt = txt.lower()
        txt = Tokenizer.tokenize(txt)
        symbols = set(['@'])
        txt = [strip_non_ascii(t,symbols) for t in txt]
        txt = ''.join([t for t in txt if len(t)>1])
        Tweets[i]['cleanTxt'] = txt
        txt = stemmer.stem(txt).split()
        Tweets[i]['nlp'] = ''.join([t for t in txt if t not in stopwords])
    return Tweets


In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re, networkx as nx, matplotlib.pyplot as plt, numpy as np 
from nltk.tokenize import TweetTokenizer; Tokenizer = TweetTokenizer(reduce_len=True) 

def strip_non_ascii(string,symbols):
    def cleanTweets(Tweets):
        ''' Return the string without non ASCII characters'''
        stripped = (c for c in string if 0 < ord(c) < 127 and c not in symbols)
        return ''.join(stripped)
    Tweets = cleanTweets(Tweets)

    for twit in Tweets:
        print("- {}".format(twit))
    

In [ ]:
timeTeks = [(t['date'], t['cleanTxt']) for t in Tweets]
for twit in timeTeks:
    print ("-{}".format(twit))

In [ ]:
from datetime import datetime

timeTeks = [((datetime.strftime(t, %a %b %d %H:%M:%S %z %Y)), txt) for t,txt in timeTeks]
for twit in timeTeks:
    print ("-{}".format(twit))
    


In [ ]:
timeTeks.sort(key=lamda)
for twit in timeTeks:
     print ("-{}".format(twit))


In [ ]:
plainTeks = [t for T, t in timeTeks]
for plain in plainTeks:
    print ("-{}".format(plain))

In [ ]:
namaFile = 'data/TeksUrutWaktu.txt'
with open(namaFile, 'w') as f:
    for tweet in plainTeks:
        f.write(tweet+'\n')
print("Simpan ke dalam File! Selesai!")

In [ ]:
import re, networkx as nx 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd, itertools, nltk
import json
from spacy.lang.id import Indonesian
from html import unescape
from unidecode import unidecode
from nltk.stem import WordNetLemmatizer
import LoadDocuments

def cleanText(T, fix={}, lang = 'en', lemma=False, stop=False, symbols_remove=False, min_charlen = 0):
    if lang == 'en':
        lemmatizer = WordNetLemmatizer()
        if stop:
            stops = set([t.strip() for t in LoadDocuments(file = 'data/stopwords_eng.txt')[0]])
        else:
            nlp_id = Indonesian()
            if stop:
                stops = set([t.strip() for t in LoadDocuments(file = 'data/stopwords_id.txt')[0]])
        if not stop:
            stops = set()
        pattern = re.compile(r'http[s]?://([a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        t = re.sub(pattern, '', T)
        t = unescape(t)
        t = fixTags(t)
        t = t.lower().strip()
        t = unidecode
        t = ''.join(''.join(s)[:2] for s in itertools.groupby(t))
        t = nltk.sent_tokenize(t)
        for i, K in enumerate(t):
            if symbols_remove:
                K = re.sub(r'[^.,a-zA-Z0-9 \n\.]', ' ',K )
            for slang, formal in fix.items():
                K = K.replace(slang,formal)
            cleanList = []
            if lang =='en':
                listKata = word_tokenize(K)
                for token in listKata:
                    if lemma:
                        token = lemmatizer.lemmatize(token)
                    if stop:
                        if len(token)>=min_charlen and token not in stops:
                            cleanList.append(token)
                        else:
                            if len(token)>=min_charlen:
                                cleanList.append(token)
                    t[i] = ' '.join(cleanList)
                else:
                    K = nlp_id(K)
                    listKata = [token.text for token in K]
                    for token in listKata:
                        if lemma:
                            token = nlp_id(token)[0].lemma_
                        if stop:
                            if len(token)>=min_charlen and token not in stops:
                                cleanList.append(token)
                        else:
                            if len(token)>=min_charlen:
                                cleanList.append(token)
                    t[i] = ' '.join(cleanList)
            return t
def fixTags(T):
    getHashtags = re.compile(r"#(\w+)")
    pisahtags = re.compile(r'[A-Z][^A-Z]*')
    t = T 
    tagS = re.findall(getHashtags, T)
    for tag in tagS:
        proper_words = ' '.join(re.findall(pisahtags, tag))
        t = t.replace('#'+tag, proper_words)
    return t 
import tweepy, sys
consumer_key = ' '
consumer_secret = ' '

qry = 'Song Jong Ki'

maxTweets = 50 
tweetsPerQry = 25 
fname='data/Hasil_Tweets_API.json'
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    sys.exit('Autentikasi gaga;, mohon cek "Consumer Key" & "Consumer Secret" twitter anda')

sinceId=None;max_id=-1;tweetCount=0
print("Mulai mengunduh maksimum {0} tweets".format(maxTweets))

with open(fname,'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets=api.search(q=qry,count=tweetsPerQry, since_id=sinceId)
            else:
                if(not sinceId):
                    new_tweets=api.search(q=qry,count=tweetsPerQry,max_id=str(max_id - 1))
                else:
                    new_tweets=api.search(q=qry,count=tweetsPerQry, since_id=sinceId)
            if not new_tweets:
                print('Tidak ada lagi Tweet ditemukan dengan Query="{0}"'.format(qry));break
            for tweet in new_tweets:
                a= tweet._json['text']
                a= ' '.join(cleanText(a, lemma=False, lang= 'id', stop=False, symbols_remove=True, min_charlen= 2,|fix={'\n\n':' '}))
                f.write(a+'\n')
            tweetCount+=len(new_tweets)
            sys.stdout.write("\r");sys.stdout.write("Jumlah Tweet telah tersimpan: %.0f" %tweetCount);sys.stdout.flush()
            max_id=new_tweets[-1].id
        except tweepy.TweepError as e:
            print("some error :" + str(e));break
print ('\nSelesai! {0} tweets tersimpan di "{1}"').format(tweetCount,fname)


In [ ]:
for twit in Tweets:
    print("-{}".format(twit))

In [ ]:
def drawGraph(G, Label=False):
    fig3 = plt.figure(); fig3.add_subplot(111)
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G,pos, alpha=0.2, node_color='blue', node_shape=600)
    if Label:
        nx.draw_networkx_labels(G,pos)
    nx.draw_networkx_edges(G,pos,width=4); plt.show()
def Graph(Tweets, Label= True):
    print("Please wait, building Graph....")
    G=nx.Graph()
    for tweet in tqdm(Tweets):
        if tweet.author not in G.nodes():
            G.add_node(tweet.author)
        mentionS = re.findall("@([a-zA-Z0-9]{1,15})", tweet['fullText'])
        for mention in mentionS:
            if "." not in mention:
                usr = mention.replace("@",'').strip()
                if usr not in G.nodes():
                    G.add_node(usr)
                G.add_edge(tweet.author,usr)
    Nn=G.number_of_nodes();Ne=G.number_of_edges()
    print('Finishedd. There are %d nodes and %d edges in the Graph.' %(Nn,Ne))
    if Label:
        drawGraph(Gm Label=True)
    else:
        drawGraph(G)
    return G
G = Graph(Tweets,Label=False)


In [ ]:
Tweets = cleanTweets(Tweets)


In [ ]:
from wordcloud import WordCloud 

def WordCloud(Tweets, file = 'wordCloud.png', plain=False):
    if plain:
        txt = Tweets
    else:
        txt = [t['nlp'] for t in Tweets]; txt = ' '.join(txt)
    wc = WordCloud(background_color="white")
    wordCloud = wc.generate(txt)
    plt.figure(num=1, facecolor='w', edgecolor='k')
    plt.imshow(wordCloud, cmap=plt.cm.jet, interpolation='nearest', aspect='auto'); plt.xticks(()); plt.yticks(())
    plt.show()

WordCloud(Tweets)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA 

def getTopics (Tweets, n_topics=5, Top_Word=7):
    Txt = [t['nlp'] for t in Tweets]
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode', token_pattern=r'\b[a-zA-Z]{3,}\b',
    max_df=0.95, min_df= 2)

    dtm_tf = tf_vectorizer.fit_transform(Txt)
    tf_terms = tf_vectorizer.get_feature_names()
    lda_tf = LDA(n_components=n_topics, learning_method='online', random_state=0).fit(dtm_tf)
    vsm_topics = lda_tf.fit_transform(dtm_tf); doc_topic = [a.argmax()+1 for a in tqdm(vsm_topics)]
    print('In total there are {0} major topics, distributed as follows'.format(len(set(doc_topic))))
    fig4 = plt.figure(); fig4.add_subplot(111)
    plt.hist(np.array(doc_topic), alpha=0.5)
    plt.show()
    print('Printing top {0} Topics, with top {1} Words:'.format(n_topics, Top_Word))
    print_Topics(lda_tf, tf_terms, n_topics, Top_Words)
    return lda_tf, dtm_tf, tf_vectorizer

def print_Topics(model, feature_names, Top_Topics, n_top_words):
    for topic_idk, topic in enumerate(model.components_[:Top_Topics]):
        print("Topic #%d:" %(topic_idkx+1))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:=n_top_words = 1:=1]]))
tf, tm, vec = getTopics(Tweets, n_topics=2, Top_Word=7)


In [ ]:
import pyLDAvis, pyLDAvis.sklearn; pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(tf, tm, vec)

In [ ]:
def sentiment(Tweets):
    print("Calculating Sentiment and Subjectivity Score:...")
    T = [translate(TextBlob(tweet['cleanTxt'])) for tweet in tqdm(Tweets)]
    Sen = [tweet.sentiment.polarity for tweet in tqdm(T)]
    Sub = [float(tweet.sentiment.Subjectivity) for tweet in tqdm(T)]
    Se, Su = [], []
    for score_se, score_su in zip(Sen_Sub):
        if score_se>0.1:
            Se.append('pos')
        elif score_se<-0.05:
            Se.append('neg')
        else:
            Se.append('net')
        if score_se>0.5:
            Su.append('Subjektif')
        else:
            Su.append('Objektif')
    label_se = ['Positif', 'Negatif', 'Netral']
    score_se = [len([True for t in Se if t =='pos']), len([True for t in Se if t=='neg']), len([True for t in Se if t =='net'])]
    label_su = ['Subjektif', 'Objektif']
    score_su = [len([True for t in Se if t =='Subjektif']), len([True for t in Se if t=='Objektif'])
    PieChart(score_se,label_se); PieChart(score_su,label_su)
    Sen = [(s,t['fullTxt']) for s, t in zip(Sen, Tweets)]
    Sen.sort(key=lambda tup: tup[0])
    Sub = [(s,t['fullTxt']) for s, t in zip(Sub, Tweets)])]
    Sub.sort(key=lambda tup: tup[0])
    return (Sen,Sub)
def translate(txt, language='en'):
    try:
        return txt.translate(to=language)
    except:
        return txt
def PieChart (score, labels):
    fig1 = plt.figure(); fig1.add.subplot(111)
    plt.pie(score, labels=labels, autopct= '%1.1f%%', startangle=140)
    plt.axis('equal')
    plt.show()
    return None
SA = sentiment(Tweets)

In [ ]:
def printSA(SA, N = 2, emo = 'positif'):
    Sen, Sub = SA
    e = emo.lower().strip()
    if e=='positif' or e=='positive':
        tweets = Sen[-N:]
    elif e=='negatif' or e=='negative':
        tweets = Sen[:N]
    elif e=='netral' or e=='neutral':
        net = [(abs(score), t) for score, t in Sen if abs(score)<0.01]
        net.sort(key=lambda tup: tup[0])
        tweets = net[:N]
    elif e=='subjektif' or e=='subjective':
        tweets = Sub[-N:]
    elif e=='objektif' or e=='objective':
        tweets = Sub[N:]
    else:
        print('Wrong function input parameter = "{0}"'.format(emo)); tweets=[]
    print('"{0}" Tweets ='.format(emo))
    for t in tweets:
        print(t)
printSA(SA, N=3, emo='positif')

In [ ]:
print("Negatif")
printSA(SA, N=2, emo = 'negatif')

print("Negatif")
printSA(SA, N=2, emo = 'netral')

print("subyektif")
printSA(SA, N=2, emo = 'subjektif')

print("obyektif")
printSA(SA, N=2, emo = 'objektif')